In [22]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime

# Path to ChromeDriver
path = "D:\\chromedriver\\chromedriver.exe"

# URL to be accessed
my_url = "https://www.imdb.com/chart/top/?ref_=nv_mv_250"

# IF YOU WANT TO DO HEADLESS BROWSER, use codes below
options = Options()
options.headless = False  # Change to True if you want to run the browser in headless mode
service = Service(path)

try:
    # THIS IS FOR NORMAL BROWSER SESSION (NOT HEADLESS)
    # Define browser to get the first URL
    browser = webdriver.Chrome(service=service, options=options)

    # Define browser to get the second/detail URL
    browser2 = webdriver.Chrome(service=service, options=options)

    # Check file availability and content is not empty
    filename = "imdb_first.csv"
    filename2 = "imdb_sec.csv"

    def visualize_data():
        print("Visualize Something")

    if os.path.isfile(filename) and os.path.getsize(filename) > 0:
        os.remove(filename)
        os.remove(filename2)

    # Initialize file handlers outside try block
    f = open(filename, "w")
    fheaders = "Name,Year,Durasi(Menit),Rating\n"
    f2 = open(filename2, "w")
    f2headers = "Name,Budget,Gross_US,Opening_Week,Open_Week_Date,Gross_World,Runtime,Color,Sound_Mix,Aspect_Ratio\n"

    f.write(fheaders)
    f2.write(f2headers)

    try:
        # ACTION 1
        # Open session for first URL
        browser.get(my_url)
        WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.sc-b189961a-0")))

        first_page = BeautifulSoup(browser.page_source, 'html.parser')
        movies = first_page.find_all("div", {"class": "sc-b189961a-0 hBZnfJ cli-children"})

        count2 = 0
        print("PRIMARY DATA")
        print("-----------------------------------")

        # Get 
        for movie in movies[:50]:
            count2 += 1
            menit = 0
            rating = "Not Rated"
            name = re.sub(r"\d+. ", "", movie.find("h3", {"class": "ipc-title__text"}).text)
            detail_line = movie.findAll("span", {"class": "sc-b189961a-8 kLaxqf cli-title-metadata-item"})
            
            if len(detail_line) > 2:
                year_movie = detail_line[0].text
                durasi = detail_line[1].text
                durasi = durasi.split(" ")
                if len(durasi) > 1:
                    menit = int(durasi[0].replace("h", "")) * 60 + int(durasi[1].replace("m", ""))
                else:
                    menit = int(durasi[0].replace("h", "")) * 60
                rating = detail_line[2].text
            elif len(detail_line) > 1:
                year_movie = detail_line[0].text
                durasi = detail_line[1].text
                durasi = durasi.split(" ")
                if len(durasi) > 1:
                    menit = int(durasi[0].replace("h", "")) * 60 + int(durasi[1].replace("m", ""))
                else:
                    menit = int(durasi[0].replace("h", "")) * 60
            else:
                year_movie = detail_line[0].text

            print("-----------------------------------")
            print(name + "," + year_movie + "," + str(menit) + "," + rating)
            f.write(name + "," + year_movie + "," + str(menit) + "," + rating + "\n")

        # ACTION 2
        links = browser.find_elements(By.XPATH, '//div[@class="sc-b189961a-0 hBZnfJ cli-children"]')

        count = 0
        print("-----------------------------------")
        print("-----------------------------------")
        print("SECONDARY DATA")
        print("panjang ", len(links))
        movie_titles = first_page.find_all("div", {"class": "ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-b189961a-9 iALATN cli-title"})

        # ambil 50 data teratas
        for link in links[:50]:
            title = re.sub(r"\d+. ", "", movie_titles[count].text)
            print("judul ke-" + str(count + 1) + ": " + title)
            count += 1
            budget_num = 0
            gross_us_num = 0
            open_week_rev_num = 0
            open_week_date_std = "1900-01-01 00:00:00"
            gross_world_num = 0

            print("----------------------")
            print(count, link.find_element(By.CSS_SELECTOR, "a").get_attribute('href'), "\n")
            browser2.get(link.find_element(By.CSS_SELECTOR, "a").get_attribute('href'))

            # BOX OFFICE DATA ON THE 2ND URL
            det_page = browser2.page_source
            container_rows = BeautifulSoup(det_page, "html.parser")
            box_office_elements = container_rows.find("div", {"data-testid": "title-boxoffice-section"})
            tech_specs_elements = container_rows.find("div", {"data-testid": "title-techspecs-section"})
            
            if box_office_elements is not None:
                det_movie = box_office_elements.find_all("span", {"class": "ipc-metadata-list-item__list-content-item"})

                if len(det_movie) > 4:
                    
                    budget = det_movie[0].text
                    gross_us = det_movie[1].text
                    open_week_rev = det_movie[2].text
                    open_week_date = det_movie[3].text            
                    gross_world = det_movie[4].text

                    print("budget = " + budget)
                    print("gross_us = " + gross_us)
                    print("opening_week_rev = " + open_week_rev)
                    print("opening_week_date = " + open_week_date)
                    print("gross_world = " + gross_world)

                    print("=====================================")
                    budget_num = int(re.sub("[A-Z£€₹$¥,()a-z]","",budget))
                    gross_us_num = int(re.sub("[A-Z£€₹$¥,()a-z]","",gross_us))
                    open_week_rev_num = int(re.sub("[A-Z£€₹$¥,()a-z]","",open_week_rev))
                    open_week_date_std = datetime.strptime(open_week_date, "%b %d, %Y")
                    gross_world_num = int(re.sub("[A-Z£€₹$¥,()a-z]","",gross_world))

                    print("-----------------------------------")
                    print("budget =", budget_num)
                    print("gross_us =" , gross_us_num)
                    print("opening_week_rev =" , open_week_rev_num)
                    print("opening_week_date =", open_week_date_std)
                    print("gross_world =", gross_world_num)

            # TECHNICAL SPECS DATA
            if tech_specs_elements is not None:
                runtime_element = tech_specs_elements.find("li", {"data-testid": "title-techspec_runtime"})
                color_element = tech_specs_elements.find("li", {"data-testid": "title-techspec_color"})
                sound_mix_element = tech_specs_elements.find("li", {"data-testid": "title-techspec_soundmix"})
                aspect_ratio_element = tech_specs_elements.find("li", {"data-testid": "title-techspec_aspectratio"})
    
                if runtime_element:
                    runtime_text = runtime_element.find("div", {"class": "ipc-metadata-list-item__content-container"})
                    runtime_text = runtime_text.text.strip() if runtime_text else "NA"
                    # Konversi runtime ke menit
                    runtime = 0
                    if 'h' in runtime_text and 'min' in runtime_text:
                        hours, minutes = map(int, re.findall(r'\d+', runtime_text))
                        runtime = hours * 60 + minutes
                    elif 'h' in runtime_text:
                        runtime = int(re.findall(r'\d+', runtime_text)[0]) * 60
                    elif 'min' in runtime_text:
                        runtime = int(re.findall(r'\d+', runtime_text)[0])
                
                color = color_element.find("div", {"class": "ipc-metadata-list-item__content-container"}).text.strip() if color_element else "NA"
                
                sound_mix_elements = sound_mix_element.find_all("li", class_="ipc-inline-list__item") if sound_mix_element else []
                if sound_mix_elements:
                    sound_mix = "; ".join([item.text.strip() for item in sound_mix_elements])
                else:
                    sound_mix = "NA"
                
                aspect_ratio_elements = aspect_ratio_element.find_all("li", class_="ipc-inline-list__item") if aspect_ratio_element else []
                if aspect_ratio_elements:
                    aspect_ratio = "; ".join([item.text.strip() for item in aspect_ratio_elements])
                else:
                    aspect_ratio = "NA"
                
                print("runtime = " + str(runtime))
                print("color = " + color)
                print("sound_mix = " + sound_mix)
                print("aspect_ratio = " + aspect_ratio)

            # Write the data even if some fields are missing
            f2.write(f"{title},{budget_num},{gross_us_num},{open_week_rev_num},{open_week_date_std},{gross_world_num},{runtime},{color},{sound_mix},{aspect_ratio}\n")

    except Exception as E:
        print(E)
    finally:
        f.close()
        f2.close()
        browser.quit()
        browser2.quit()
except Exception as ex:
    print("Failed to start browser or driver:", ex)

C:\Users\tiwi\AppData\Local\Temp\ipykernel_3924\1684264724.py:26: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = False  # Change to True if you want to run the browser in headless mode


PRIMARY DATA
-----------------------------------
-----------------------------------
The Shawshank Redemption,1994,142,R
-----------------------------------
The Godfather,1972,175,R
-----------------------------------
The Dark Knight,2008,152,PG-13
-----------------------------------
The Godfather Part II,1974,202,R
-----------------------------------
Angry Men,1957,96,Approved
-----------------------------------
Schindler's List,1993,195,R
-----------------------------------
The Lord of the Rings: The Return of the King,2003,201,PG-13
-----------------------------------
Pulp Fiction,1994,154,R
-----------------------------------
The Lord of the Rings: The Fellowship of the Ring,2001,178,PG-13
-----------------------------------
The Good, the Bad and the Ugly,1966,178,R
-----------------------------------
Forrest Gump,1994,142,PG-13
-----------------------------------
The Lord of the Rings: The Two Towers,2002,179,PG-13
-----------------------------------
Fight Club,1999,139,R
--------

In [23]:
# MERGE 2 IMDB_DATA CSV FILES

import pandas as pd

fn1 = 'imdb_first.csv'
fn2 = 'imdb_sec.csv'

fn3 = 'imdb_combined.csv'

if os.path.isfile(fn3) and os.path.getsize(fn3) > 0:      
    os.remove(fn3)
    
# Read CSV files into dataframes
df1 = pd.read_csv(fn1)
df2 = pd.read_csv(fn2)

merged = df1.merge(df2,on='Name')
merged.to_csv("imdb_combined.csv", index=False)